Import Our Libraries

In [1]:
import openpyxl
import json
from pathlib import Path
import os
from typing import List
import openpyxl.workbook
import pandas as pd
import re

Define Our Parsing Functions

In [2]:


def load_workbook(filepath: Path) -> openpyxl.workbook.workbook.Workbook:
    if filepath.suffix != ".xlsx":
        raise ValueError(
            f"The inputted file is of type {filepath.suffix}. Please provide a .xlsx file."
        )

    try:
        workbook = openpyxl.load_workbook(filepath)

    except Exception as e:
        raise f"An unexpected error occurred: {e}"

    else:
        return workbook


def load_worksheet(
    workbook: openpyxl.workbook.workbook.Workbook, sheet_name: str
) -> openpyxl.worksheet.worksheet.Worksheet:
    if sheet_name not in workbook.sheetnames:
        raise KeyError(f"Sheet '{sheet_name}' cannot be found in workbook.")

    else:
        return workbook[sheet_name]


def parse_sheet_data(worksheet: openpyxl.worksheet.worksheet.Worksheet) -> List[List]:
    tables = []
    current_table = []
    for row in worksheet.iter_rows(values_only=True):
        if all(cell is None for cell in row):
            if current_table:
                tables.append(current_table)
                current_table = []

        else:
            current_table.append([r for r in row if r is not None])

    if current_table:
        tables.append(current_table)

    return tables


def parse_table_data(table: List[List]) -> dict:
    if len(table) < 2:
        return None

    column_name, column_description = table.pop(0)
    column_type = table.pop(0)[1]
    encodings = {key: value for key, value in table if key is not None}

    return {
        "field_code": column_name,
        "field_text": column_description,
        "field_type": column_type,
        "encodings": encodings,
    }


def load_question_schema(filepath: Path, sheet_name: str) -> List[dict]:
    wb = load_workbook(filepath=filepath)
    sheet = load_worksheet(workbook=wb, sheet_name=sheet_name)
    processed_tables = parse_sheet_data(sheet)

    return [parse_table_data(t) for t in processed_tables]

Load Our Data and Drop the encodings column, for now.

In [3]:
file_path = Path(os.path.abspath("")).parent / "survey.xlsx"
json_output = load_question_schema(filepath=file_path, sheet_name="Questions")
survey_df = pd.DataFrame.from_records(json_output)#.drop(columns=["encodings"])
survey_df.head(10)

,field_code,field_text,field_type,encodings
0,trans_id,Record ID,text,{}
1,status,Status,radio,"{'C': 'Completed', 'G': 'Targetable quota', 'Q..."
2,resp_start_ts,Respondent start timestamp,date,{}
3,resp_last_ts,Respondent last timestamp,date,{}
4,language,Survey language,text,{}
5,testmode,Test mode,radio,"{'0': 'No', '1': 'Level 1', '1.1': 'Level 1, s..."
6,panel,Panel,radio,"{'0': 'No Panel Assigned', '9999': 'Panel Not ..."
7,base_country,Base Country,text,{}
8,QGLG_SAMPLE,GLG sample source,radio,"{'1': 'online', '2': 'CATI'}"
9,QCOUNTRY,Country,radio,"{'1': 'UK', '2': 'Germany', '3': 'France', '4'..."


In [ ]:
other_match_pattern = re.compile(r".*_97_OTH(_english_uk)?")
# row_match_pattern = re.compile(r".*R\d+(_97_OTH)?(_english_uk)?")

# Find all the primary keyts found in the data set.
for key in survey_df["column_name"].values:
    if not key.startswith("Q"):
        # Filter out non-question fields and process them later.
        print(f"Non-Question key {key}.")

    else:
        # Skip fields that are text boxes or translations (They end in _97_OTH[_english_uk])
        if re.match(other_match_pattern, key):
            pass

        # If key ends with R#, then it is a row of a table and we can extract the primary key./
        elif re.match(r"(R\d+)$", key):
            pass
            # primary_key = 
            



Separate our data by columns that begin with Q

In [4]:
question_fields = survey_df.loc[survey_df["field_code"].str.startswith("Q")]
question_fields.head(50)

,field_code,field_text,field_type,encodings
8,QGLG_SAMPLE,GLG sample source,radio,"{'1': 'online', '2': 'CATI'}"
9,QCOUNTRY,Country,radio,"{'1': 'UK', '2': 'Germany', '3': 'France', '4'..."
10,QCOUNTRY_OVER,Please select the country for which you take t...,radio,"{'1': 'UK', '2': 'Germany', '3': 'France', '4'..."
11,QCOUNTRY_QUO,QUOTA | country,quotas,"{'1': 'UK', '2': 'Germany', '3': 'France', '4'..."
12,QPHASE,main vs. boost,radio,"{'1': 'main', '2': 'boost'}"
13,QREV,Revisions,radio,"{'10': 'Launch source', '20': '4/12/24 Fri am ..."
14,QPHASE_QUO,QUOTA | rep vs. boost (main / completes),quotas,"{'1': 'main', '2': 'boost'}"
15,QSEMPLOYMENT,What is your employment status?,radio,"{'1': 'Full-time', '2': 'Part-time', '3': 'Con..."
16,QSINDUSTRY,How would you classify the industry your compa...,radio,"{'1': 'Real estate', '2': 'Industrials', '3': ..."
17,QSINDUSTRY_97_OTH,Other,text,{}


Write a function to separate the field name from our codes.
Must account for those that are QMKT<\FIELDNAME\> as well.

In [5]:
def field_code_processor(series: str) -> list[str, bool]:
    code = series["field_code"]
    if "MKT" in code:
        _, field_name = code.split("MKT")
        mkt_flag = True 

    else:
        field_name = code[1:]
        mkt_flag = False

    return [field_name, mkt_flag]

In [6]:
processed_codes = question_fields[["field_code"]].apply(lambda row: field_code_processor(row), axis=1, result_type="expand")
question_fields.loc[:, "field_name"] = processed_codes[0].values
question_fields.loc[:, "mkt_flag"] = processed_codes[1].astype("bool").values
question_fields.head(50)

C:\Users\ollie\AppData\Local\Temp\ipykernel_25228\743184897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_fields.loc[:, "field_name"] = processed_codes[0].values
C:\Users\ollie\AppData\Local\Temp\ipykernel_25228\743184897.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_fields.loc[:, "mkt_flag"] = processed_codes[1].astype("bool").values


,field_code,field_text,field_type,encodings,field_name,mkt_flag
8,QGLG_SAMPLE,GLG sample source,radio,"{'1': 'online', '2': 'CATI'}",GLG_SAMPLE,False
9,QCOUNTRY,Country,radio,"{'1': 'UK', '2': 'Germany', '3': 'France', '4'...",COUNTRY,False
10,QCOUNTRY_OVER,Please select the country for which you take t...,radio,"{'1': 'UK', '2': 'Germany', '3': 'France', '4'...",COUNTRY_OVER,False
11,QCOUNTRY_QUO,QUOTA | country,quotas,"{'1': 'UK', '2': 'Germany', '3': 'France', '4'...",COUNTRY_QUO,False
12,QPHASE,main vs. boost,radio,"{'1': 'main', '2': 'boost'}",PHASE,False
13,QREV,Revisions,radio,"{'10': 'Launch source', '20': '4/12/24 Fri am ...",REV,False
14,QPHASE_QUO,QUOTA | rep vs. boost (main / completes),quotas,"{'1': 'main', '2': 'boost'}",PHASE_QUO,False
15,QSEMPLOYMENT,What is your employment status?,radio,"{'1': 'Full-time', '2': 'Part-time', '3': 'Con...",SEMPLOYMENT,False
16,QSINDUSTRY,How would you classify the industry your compa...,radio,"{'1': 'Real estate', '2': 'Industrials', '3': ...",SINDUSTRY,False
17,QSINDUSTRY_97_OTH,Other,text,{},SINDUSTRY_97_OTH,False


Now select all of our tabular questions based on ReGex.
<br>From my exploration, any tabular style question should follow these rules:
<li><t>Contains a row identified R#, where # is any number.<br>
<li><t>May end in _97_OTH or _english_uk


In [7]:
row_match_pattern = re.compile(r".*R\d+(_97_OTH)?(_english_uk)?")
tabular_questions = question_fields[question_fields["field_name"].str.match(row_match_pattern, case=True)]
tabular_questions.head(25)

,field_code,field_text,field_type,encodings,field_name,mkt_flag
40,QSSOFTWARER1,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER1,False
41,QSSOFTWARER2,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER2,False
42,QSSOFTWARER3,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER3,False
43,QSSOFTWARER4,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER4,False
44,QSSOFTWARER5,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER5,False
45,QSSOFTWARER6,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER6,False
46,QSSOFTWARER7,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER7,False
47,QSSOFTWARER8,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER8,False
48,QSSOFTWARER9,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER9,False
49,QSSOFTWARER10,"Which, if any of the following features of pro...",checkbox,"{'1': 'Manually/basic software (e.g., with ema...",SSOFTWARER10,False


In [10]:
look_behind = re.compile(r"(\w+)(?=R\d+)")
tabular_questions.loc[:, "clean_field_name"] = tabular_questions.loc[:, "field_name"].str.extract(look_behind)[0]

In [41]:
split_text = tabular_questions.loc[:, "field_text"].str.split(r" \| ", expand=True, n=1)
tabular_questions.loc[:, "field_question"] = split_text.loc[:, 0]
tabular_questions.loc[:, "sub_field_text"] = split_text.loc[:, 1]


In [ ]:
clean_table_codes = tabular_questions["clean_field_name"].unique()
# qtable = dict()
# for f in unique_fields:
#     sub_df = tabular_questions.loc[tabular_questions["clean_field_name"] == f]
#     # first_code = sub_df["field_code"].iloc[0]
#     mappings = sub_df["encodings"].iloc[0]
    # print(f, json.dumps(mappings, indent=2), sub_df.head(), sep="\n")

In [51]:
samples = ['GLG_SAMPLE', 'COUNTRY', 'COUNTRY_OVER', 'COUNTRY_QUO', 'PHASE', 'REV',
 'PHASE_QUO', 'SEMPLOYMENT', 'SINDUSTRY', 'SINDUSTRY_97_OTH',
 'SINDUSTRY_97_OTH_english_uk', 'SREALESTATE', 'SPROPERTY',
 'SPROPERTY_97_OTH', 'SPROPERTY_97_OTH_english_uk', 'SPROPERTY2',
 'SPROPERTY3', 'SPROPERTY3_97_OTH', 'SPROPERTY3_97_OTH_english_uk',
 'SLOCATION', 'SLOCATION_97_OTH', 'SLOCATION_97_OTH_english_uk', 'SUNITS',
 'SUNITS_GROUPS', 'SUNITS_GROUPS']

other_match_pattern = re.compile(r".*_97_OTH(_english_uk)?")
row_match_pattern = re.compile(r".*R\d+(_97_OTH)?(_english_uk)?")

for n in samples:
    # Check for MKT quotas
    if "MKT" in n:
        pass

    elif n.endswith("GROUPS"):
        pass

    elif re.match(other_match_pattern, n):
        pass

    elif re.match(row_match_pattern, n):
        pass

    else:
        print(n)

    

GLG_SAMPLE
COUNTRY
COUNTRY_OVER
COUNTRY_QUO
PHASE
REV
PHASE_QUO
SEMPLOYMENT
SINDUSTRY
SREALESTATE
SPROPERTY
SPROPERTY2
SPROPERTY3
SLOCATION
SUNITS
